In [17]:
# Dependencia
!pip install findspark
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
import findspark, datetime, pytz
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lit,row_number,col,count

findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [19]:
agora = datetime.datetime.now(pytz.timezone('America/Sao_paulo'))
strBronzePath = f'../BRONZE/loja_simulada-vendas'
# /home/cj/lisarbjc/lab/lake/users/root/CASE-ingestao-gold.ipynb
# [PATH_NOT_FOUND] Path does not exist: file:/home/cj/lisarbjc/lab/lake/users/root/BRONZE/loja_simulada-vendas-clientes.

In [20]:
import os 
dir_path = os.getcwd()
print(dir_path)

/home/cj/lisarbjc/lab/lake/root


In [21]:
# Create SparkSession
spark = (SparkSession.builder
           .appName('IngestaoBronze')
           .config("packages", "org.apache.spark:mysql-connector-java-8.0.13.jar")
           .getOrCreate()
        )#.setLevel("OFF")
spark.sparkContext.setLogLevel("OFF")

In [22]:
def pegarBronzeAtual(tabela):
    try:
        tempDF = (spark.read.parquet(f'{strBronzePath}-{tabela}')).cache()
        print(f'Base BRONZE "{tabela}" encontrada!')
        return tempDF
    except Exception as e:
        print(f'Base BRONZE "{tabela}" NÃO encontrada!\n', e)
        return None

In [23]:
def enviarAlerta(tipoAlerta, mensagem):
    print(F'Encaminhando alerta de {tipoAlerta}')
    print(f'{tipoAlerta} - type: {type(mensagem)} - Mensagem:', mensagem)
    return

In [24]:
# Read from MySQL Table
def pegarDadosDaTable(tabela):
    try:
        return (spark.read
                .format("jdbc")
                .option("driver","com.mysql.cj.jdbc.Driver")
                .option("url", "jdbc:mysql://localhost:3306/vendas?allowPublicKeyRetrieval=true&useSSL=false")
                .option("dbtable", tabela)
                .option("user", "big_data_importer")
                .option("password", "big_data_importer")
                .load()
             )
    except:
        enviarAlerta('Erro', f'Não foi possível se conectar ao MYSQL - tabela "{tabela}"')
        return None

In [25]:
def salvarFinalDF(df,tabela):
    return df.write.mode("overwrite").parquet(f'{strBronzePath}-{tabela}')

In [30]:
def clientes():
    tabela = 'clientes'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
#     baseNovaTempDF.show(1, vertical=True)
    baseNovaTempDF.printSchema()
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
   # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.cliente == baseNovaDF.cliente) &
                (bronzeAtualDF.estado == baseNovaDF.estado) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.sexo == baseNovaDF.sexo) &
                (bronzeAtualDF.status == baseNovaDF.status))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_cliente")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
#         finalDF.show(1,vertical=True)
        finalDF.printSchema()
    else:
        windowSpec = Window.orderBy("id_cliente")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return
clientes()

Base BRONZE "clientes" encontrada!
root
 |-- id_cliente: integer (nullable = true)
 |-- cpf: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- email: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- origem_racial: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- status: string (nullable = true)

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze clientes: 250
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 0
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 0
root
 |-- id_cliente: integer (nullable = true)
 |-- cpf: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- email: string (nullable = true)
 |-- cliente: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- origem_racial: string (nullable = true)
 |-- sexo

In [11]:
def vendedores():
    tabela = 'vendedores'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.cpf == baseNovaDF.cpf) &
                (bronzeAtualDF.telefone == baseNovaDF.telefone) &
                (bronzeAtualDF.email == baseNovaDF.email) &
                (bronzeAtualDF.origem_racial == baseNovaDF.origem_racial) &
                (bronzeAtualDF.nome == baseNovaDF.nome) )
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_vendedor")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_vendedor")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [12]:
def produtos():
    tabela = 'produtos'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.produto == baseNovaDF.produto) &
                (bronzeAtualDF.preco == baseNovaDF.preco))
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_produto")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_produto")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [13]:
def vendas():
    tabela = 'vendas'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.id_vendedor == baseNovaDF.id_vendedor) &
                (bronzeAtualDF.id_cliente == baseNovaDF.id_cliente) &
                (bronzeAtualDF.data == baseNovaDF.data)) &
                (bronzeAtualDF.total == baseNovaDF.total)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy("id_venda")        
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [14]:
def itens_venda():
    tabela = 'itens_venda'
    bronzeAtualDF = pegarBronzeAtual(tabela)
    baseNovaTempDF = pegarDadosDaTable(tabela)
    if baseNovaTempDF == None:
        return
    baseNovaDF = baseNovaTempDF.withColumn("data_carga", lit(agora.strftime("%Y-%m-%d")).cast('date'))
    enviarAlerta('Informacao', f'COUNT bronze {tabela}: {baseNovaDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    if bronzeAtualDF != None:
    # SELECIONAR DADOS NOVOS
        # bronzeAtualDF = bronzeAtualDF1  # .select('*').where(col('id_venda') < 10)  # TODO inserida para teste, retirar
        apenasNovosDF = (baseNovaDF.join(bronzeAtualDF          
            ,((bronzeAtualDF.id_produto == baseNovaDF.id_produto) &
                (bronzeAtualDF.id_venda == baseNovaDF.id_venda) &
                (bronzeAtualDF.quantidade == baseNovaDF.quantidade) &
                (bronzeAtualDF.valor_unitario == baseNovaDF.valor_unitario)) &
                (bronzeAtualDF.valor_total == baseNovaDF.valor_total) &
                (bronzeAtualDF.desconto == baseNovaDF.desconto)
            ,"leftanti"))
        enviarAlerta('Informacao', f'COUNT novos registros: {apenasNovosDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    # ENCONTRAR ULTIMA SURROGATE KEY
        maxSK = bronzeAtualDF.selectExpr("max(surrogate_key) as max_key").collect()[0]["max_key"]
    # ADICIONAR SK AO NOVO DF
        windowSpec = Window.orderBy(*["id_venda", "id_produto"])
        apenasNovosSKDF = (apenasNovosDF.withColumn("surrogate_key", maxSK + row_number().over(windowSpec)))
    # FINAL DO PROCESSO
        finalDF = bronzeAtualDF.unionByName(apenasNovosSKDF)
        salvarFinalDF(finalDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {finalDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    else:
        windowSpec = Window.orderBy("id_venda")
        bronzeNovoDF = baseNovaDF.withColumn("surrogate_key", row_number().over(windowSpec))
        salvarFinalDF(bronzeNovoDF, tabela)
        enviarAlerta('Informacao',
            f'Registros a serem salvos: {bronzeNovoDF.select(count("*").alias("QTD")).collect()[0]["QTD"]}')
    return

In [15]:
def main():
    try:
        enviarAlerta('Informacao', 'ingestão para base BRONZE iniciada...')
        clientes()
        vendedores()
        produtos()
        vendas()
        itens_venda()
        enviarAlerta('Sucesso', 'Ingestão para base BRONZE concluída!!!')
    except Exception as e:
        enviarAlerta('Erro', e)
    finally:
        spark.stop()
        print('Conexão spark finalizada!')

In [16]:
main()

Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: ingestão para base BRONZE iniciada...
Base BRONZE "clientes" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze clientes: 250
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 0
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 0
Base BRONZE "vendedores" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze vendedores: 10
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT novos registros: 0
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: Registros a serem salvos: 10
Base BRONZE "produtos" encontrada!
Encaminhando alerta de Informacao
Informacao - type: <class 'str'> - Mensagem: COUNT bronze produtos: 10
Encaminhando alerta de Informacao
I